In [1]:
import os, pickle, sys
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import glob
from prettytable import PrettyTable
from tqdm import tqdm
from torch.utils.data import Dataset

import torch
from torch import nn
from torch.nn import Linear,Sequential
from torch.utils.data import DataLoader
import torch.nn.functional as func
import time
from math import isnan,isinf,log,exp
import random

import tensorflow as tf

In [2]:
fname=r'../myp/tensors.p'
rname='CIFAR10'

In [3]:
runs=[]
f = open(fname,'rb')
while(1):
    try:
        runs.append(pickle.load(f))
    except EOFError:
        break
f.close()
print(fname, len(runs))

../myp/tensors.p 15625


In [4]:
# len(runs)

In [5]:
# len(runs[0]['logmeasures']['synflow'])

In [6]:
# runs[0]['logmeasures']['synflow']

In [37]:
x_data=[]
y_data=[]
for idx in range(len(runs)):
    temp=runs[idx]['logmeasures']['synflow']
    for i in range(len(temp)):
        temp[i]=temp[i].item()
    temp=torch.tensor(temp,dtype=torch.float32)
    x_data.append(temp)
    y_data.append(runs[idx]['testacc'])

In [38]:
x_data=torch.tensor([item.cpu().detach().numpy() for item in x_data],dtype=torch.float64).cuda()

In [41]:
x_data[0][1]==0

tensor(True, device='cuda:0')

In [43]:
for i in range(len(x_data)):
    indices = torch.sort(x_data[i],stable=True).indices
    #对索引排序，获得原本元素的排名
    indices=torch.sort(indices,stable=True).indices
    for j in range(len(x_data[i])):
        if x_data[i][j]==0:
            continue
        x_data[i][j]=indices[j]

In [44]:
x_data[0]

tensor([184.,   0.,   0.,   0., 158.,   0.,   0.,   0., 155.,   0.,   0.,   0.,
          0.,   0.,   0.,   0., 154.,   0.,   0.,   0., 163.,   0.,   0.,   0.,
          0.,   0.,   0.,   0., 161.,   0.,   0.,   0., 156.,   0.,   0.,   0.,
          0.,   0.,   0.,   0., 160.,   0.,   0.,   0., 159.,   0.,   0.,   0.,
          0.,   0.,   0.,   0., 157.,   0.,   0.,   0., 162.,   0.,   0.,   0.,
          0., 185., 152., 153.,   0.,   0.,   0., 172.,   0.,   0.,   0., 165.,
          0.,   0.,   0.,   0.,   0.,   0.,   0., 171.,   0.,   0.,   0., 164.,
          0.,   0.,   0.,   0.,   0.,   0.,   0., 166.,   0.,   0.,   0., 173.,
          0.,   0.,   0.,   0.,   0.,   0.,   0., 170.,   0.,   0.,   0., 168.,
          0.,   0.,   0.,   0.,   0.,   0.,   0., 167.,   0.,   0.,   0., 169.,
          0.,   0.,   0.,   0., 186., 150., 151.,   0.,   0.,   0., 182.,   0.,
          0.,   0., 175.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 176.,   0.,
          0.,   0., 180.,   0.,   0.,   

In [45]:
y_data=torch.tensor(y_data,dtype=torch.float64)

In [46]:
y_data=y_data/100

In [47]:
trainx=[]
trainy=[]

In [48]:
for i in range (len(x_data)):
    if isinf(torch.sum(x_data[i]).item()):
        continue
    trainx.append(x_data[i])
    trainy.append(y_data[i])

In [49]:
trainx=torch.tensor([item.cpu().detach().numpy() for item in trainx],dtype=torch.float64)

In [50]:
trainy=torch.tensor(trainy,dtype=torch.float64)

In [51]:
print(len(trainx),len(trainy))

15625 15625


In [52]:
trainx=trainx.cuda()
trainy=trainy.cuda()

In [53]:
#这句话很关键，nn默认类型为float32
torch.set_default_dtype(torch.float64)

In [69]:
class zyt(nn.Module):
    def __init__(self):
        super(zyt,self).__init__()
        self.model1=Sequential(
#             nn.BatchNorm1d(188),
#            nn.LayerNorm(188),
            Linear(188,128),
#             nn.LayerNorm(128),
            nn.ReLU(),
            Linear(128,64),
            nn.ReLU(),
            Linear(64,10),
            nn.ReLU(),
            
            Linear(10,1),
            nn.Sigmoid()
        )
    
    def forward(self,x):
        x=self.model1(x)
        return x
zyt1=zyt()
zyt1=zyt1.cuda()

In [70]:
#损失函数
class SpearmanLossFunc(nn.Module):
    def __init__(self):
        super(SpearmanLossFunc,self).__init__()

    def forward(self, t1,t2):
        t1=t1.cpu().detach().numpy()
        t2=t2.cpu().detach().numpy()
        loss = -10*abs(stats.spearmanr(t1,t2,nan_policy='omit').correlation)
        loss=torch.tensor(loss,requires_grad=True)
        return loss
    
class MyLoss(nn.Module):
    def __init__(self):
        super(MyLoss,self).__init__()

    def forward(self, t1,t2):
        t1=t1.cpu().detach().numpy()
        t2=t2.cpu().detach().numpy()
        loss=0
        sample_size=500
        sample=random.sample(range(len(trainx)),sample_size)
        t1=t1.reshape(-1,)
        for i in range(sample_size):
            for j in range(sample_size):
                if i==j:
                    continue
                loss+=log(1+exp(-np.sign((t1[sample[i]]-t1[sample[j]])*(t2[sample[i]]-t2[sample[j]]))))
        loss=loss/(sample_size*(sample_size-1))
        loss=torch.tensor(loss,requires_grad=True)
        return loss

In [71]:
# loss_fn=torch.nn.MSELoss()
loss_fn=SpearmanLossFunc()
loss_fn=loss_fn.cuda()
#优化器
learning_rate=0.001
optimizer=torch.optim.Adam(zyt1.parameters(),lr=learning_rate)
#训练的轮数
epoch=50

In [72]:
final_output=[]
for i in range(epoch):
    zyt1.train()
    print("------第{}轮训练开始-------".format(i+1))
#     start_time=time.time()
    total_train_loss=0
    output=zyt1(trainx)
    print(output)
#     print(abs(stats.spearmanr(output.cpu().detach().numpy(),trainy.cpu().detach().numpy(),nan_policy='omit').correlation))
    loss=loss_fn(output,trainy)
    #优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()                
    print("loss:{}".format(loss.item()))
#     end_time=time.time()
#     print("time_cost:{}".format(end_time-start_time))

------第1轮训练开始-------
tensor([[0.5110],
        [0.7284],
        [0.1835],
        ...,
        [0.2986],
        [0.6453],
        [0.0451]], device='cuda:0', grad_fn=<SigmoidBackward0>)
loss:-1.2268932108497936
------第2轮训练开始-------
tensor([[0.5110],
        [0.7284],
        [0.1835],
        ...,
        [0.2986],
        [0.6453],
        [0.0451]], device='cuda:0', grad_fn=<SigmoidBackward0>)
loss:-1.2268932108497936
------第3轮训练开始-------
tensor([[0.5110],
        [0.7284],
        [0.1835],
        ...,
        [0.2986],
        [0.6453],
        [0.0451]], device='cuda:0', grad_fn=<SigmoidBackward0>)
loss:-1.2268932108497936
------第4轮训练开始-------
tensor([[0.5110],
        [0.7284],
        [0.1835],
        ...,
        [0.2986],
        [0.6453],
        [0.0451]], device='cuda:0', grad_fn=<SigmoidBackward0>)
loss:-1.2268932108497936
------第5轮训练开始-------
tensor([[0.5110],
        [0.7284],
        [0.1835],
        ...,
        [0.2986],
        [0.6453],
        [0.0451]], devic

In [73]:
abs(stats.spearmanr(output.cpu().detach().numpy(),trainy.cpu().detach().numpy(),nan_policy='omit').correlation)

0.12268932108497937

In [74]:
output

tensor([[0.5110],
        [0.7284],
        [0.1835],
        ...,
        [0.2986],
        [0.6453],
        [0.0451]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [75]:
trainy

tensor([0.8152, 0.9051, 0.8184,  ..., 0.8944, 0.6579, 0.8401], device='cuda:0')

[11317,
 12368,
 5271,
 1256,
 15302,
 9442,
 11171,
 1173,
 14640,
 14189,
 3138,
 10664,
 3379,
 2404,
 5072,
 10564,
 5248,
 5505,
 497,
 12080,
 15159,
 9967,
 14833,
 13409,
 12849,
 15042,
 14520,
 6761,
 13199,
 1803,
 7594,
 5934,
 12617,
 5005,
 10352,
 9152,
 2142,
 7127,
 11549,
 12529,
 12787,
 11464,
 7077,
 11650,
 11367,
 11129,
 2957,
 3293,
 14554,
 4071,
 6842,
 14135,
 4034,
 9183,
 1530,
 3164,
 1186,
 10957,
 2856,
 1229,
 10618,
 3382,
 4814,
 5756,
 14454,
 11649,
 4993,
 3276,
 8420,
 3768,
 6261,
 7703,
 12473,
 8133,
 7830,
 14479,
 13746,
 14318,
 7551,
 1034,
 10662,
 8413,
 13393,
 2892,
 12284,
 12558,
 12712,
 10594,
 10475,
 2452,
 7502,
 9830,
 14632,
 8811,
 630,
 6149,
 455,
 977,
 14731,
 3076,
 14686,
 9214,
 13869,
 13168,
 15086,
 15207,
 3855,
 12261,
 949,
 4090,
 9075,
 12305,
 2679,
 5729,
 8764,
 4166,
 4180,
 3661,
 1732,
 14092,
 13559,
 11391,
 2129,
 7088,
 12207,
 1332,
 9921,
 3863,
 15258,
 11172,
 946,
 12443,
 10623,
 12527,
 15171,
